# 卷积神经网络

 ## 1 简单的卷积网络
 ### 1.1 卷积模块

In [1]:
import torch
import numpy as np
from torch import nn, optim
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

In [3]:
# 定义简单的卷积网络模型
class SimpleCNN(nn.Module):
    def __init__(self):            # 定义网络结构
        super().__init__()         # 输入 [batch_size,3,32,32] 3表示深度
        layer1 = nn.Sequential()   # 叠加第1个网络层，卷积层
        # (in_channels, out_channels, kernel_size,stride,padding)
        layer1.add_module('conv1', nn.Conv2d(32, 3, 1, padding=1))
        # get [batch_size, 32, 32, 32]
        layer1.add_module('relu1', nn.ReLU(True))
        layer1.add_module('pool1', nn.MaxPool2d(2,2))
        # get [batch_size, 32, 16, 16]
        self.layer1 = layer1
        
        layer2 = nn.Sequential()   # 定义第2个网络层
        layer2.add_module('conv2', nn.Conv2d(32, 64, 3, 1, padding=1))
        # get [batch_size, 64, 16, 16] 
        layer2.add_module('relu2', nn.ReLU(True))
        layer2.add_module('pool2', nn.MaxPool2d(2,2))
        # get [batch_size, 64, 8, 8]
        self.layer2 = layer2
        
        layer3 = nn.Sequential()   # 定义第3个网络层
        layer3.add_module('conv3', nn.Conv2d(64, 128, 3, 1, padding=1))
        # get [batch_size, 128, 8, 8]
        layer3.add_module('relu3', nn.ReLU(True))
        layer3.add_module('pool3', nn.MaxPool2d(2,2))
        # get [batch_size, 128, 4, 4]
        self.layer3 = layer3
        
        layer4 = nn.Sequential()   # 定义第4个网络层，全连接层
        layer4.add_module('fc1', nn.Linear(128*4*4, 512))
        # get [batch_size, 512]
        layer4.add_module('fc_relu1', nn.ReLU(True))
        layer4.add_module('fc2', nn.Linear(512, 64))
        # get [batch_size, 64]
        layer4.add_module('fc2_relu2', nn.ReLU(True))
        layer4.add_module('fc3', nn.Linear(64, 10))
        # get [batch_size, 10]
        self.layer4 = layer4
    def forward(self, x):
        conv1 = self.layer1(x)   # 前3层为卷积层
        conv2 = self.layer2(conv1)
        conv3 = self.layer3(conv2)
        # 注意：全连接前要把数据的维度将为两维，-1维度长*宽*深度
        fc_input = conv3.view(conv3.size(0), -1)  # reshape为 [batch_size, -1]
        fc_out = self.layer4(fc_input)
        return fc_out
model = SimpleCNN()
print(model)

SimpleCNN(
  (layer1): Sequential(
    (conv1): Conv2d(32, 3, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU(inplace)
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace)
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (fc1): Linear(in_features=2048, out_features=512, bias=True)
    (fc_relu1): ReLU(inplace)
    (fc2): Linear(in_features=512, out_features=64, bias=True)
    (fc2_relu2): ReLU(inplace)
    (fc3): Linear(in_features=64, out_features=10, bias=True)
  )
)


小结:
- 模型层顺序：CNN+BN+ReLU+Pooling，先激活再池化;FC+ReLU
- 输出层不用激活函数

### 1.2 提取模型的层结构

|nn.Module属性|功能|实例|
|-|-|-|
|children()|返回下一级迭代器|self.layer1|
|modules()|返回所有模块迭代器|self.layer1.conv1|
|named_children()|返回模块的名称|其他功能同上|
|named_modules()|返回模块的名称|其他功能同上|

In [33]:
# 提取已有模型中的结构
new_model = nn.Sequential(*list(model.children())[:2])   # *代表使用可变参数
print(new_model)
# list(model.children())[0]

Sequential(
  (0): Sequential(
    (conv1): Conv2d(32, 3, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU(inplace)
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)


In [46]:
# 提取模型中所有的卷积层
conv_model = nn.Sequential()
for layer in model.named_modules():    # layer[0]层名称, layer[1]层类型
#     print(layer)

    if isinstance(layer[1], nn.Conv2d):  
        print(layer)
        print('-'*50)
        conv_model.add_module(layer[0].split('.')[-1], layer[1])
print('1.新网络的结构:')
print(conv_model)

('layer1.conv1', Conv2d(32, 3, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1)))
--------------------------------------------------
('layer2.conv2', Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
--------------------------------------------------
('layer3.conv3', Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
--------------------------------------------------
1.新网络的结构:
Sequential(
  (conv1): Conv2d(32, 3, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


### 1.3 提取参数及初始化

|nn.Module属性|功能|
|-|-|
|parameters()|返回全部参数的迭代器|
|named_parameters()|返回模块的名称，其他功能同上|

In [64]:
# 提取模型的所有参数
for param in model.named_parameters():
    # param[0]:层名称.weight/bias  param[1]:参数值，权重是Variable
    print(param[0])
    print((param[1]).data.shape)
    print('-'*50)

layer1.conv1.weight
torch.Size([3, 32, 1, 1])
--------------------------------------------------
layer1.conv1.bias
torch.Size([3])
--------------------------------------------------
layer2.conv2.weight
torch.Size([64, 32, 3, 3])
--------------------------------------------------
layer2.conv2.bias
torch.Size([64])
--------------------------------------------------
layer3.conv3.weight
torch.Size([128, 64, 3, 3])
--------------------------------------------------
layer3.conv3.bias
torch.Size([128])
--------------------------------------------------
layer4.fc1.weight
torch.Size([512, 2048])
--------------------------------------------------
layer4.fc1.bias
torch.Size([512])
--------------------------------------------------
layer4.fc2.weight
torch.Size([64, 512])
--------------------------------------------------
layer4.fc2.bias
torch.Size([64])
--------------------------------------------------
layer4.fc3.weight
torch.Size([10, 64])
--------------------------------------------------
layer

In [73]:
# 提取参数并初始化????????
init = nn.init
for m in model.modules():  # 访问所有模块
    if isinstance(m, nn.Conv2d):
        init.normal_(m.weight.data)
        init.xavier_normal_(m.weight.data)
        init.kaiming_normal_(m.weight.data)
        m.bias.data.fill_(0)
    elif isinstance(m, nn.Linear):
        m.weight.data.normal_()